### Test other model

In [42]:
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def ypred_ytrue(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)
    return predictions, labels

def compute_metrics(eval_pred):
    predictions, labels = ypred_ytrue(eval_pred)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

def evaluation_metric(pred):
    y_pred, y_true = ypred_ytrue(pred)

    confusion = confusion_matrix(y_true, y_pred)
    print('Confusion Matrix\n')
    print(confusion)

    #one-vs-rest way confusion matrix
    print('\nMultilabel Confusion Matrix\n')
    print(multilabel_confusion_matrix(y_true, y_pred,labels=[0, 1, 2,3,4,5]))
    
    #accuracy_score, precision_score, recall_score, f1_score
#     print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_true, y_pred)))

#     print('Micro Precision: {:.2f}'.format(precision_score(y_true, y_pred, average='micro')))
#     print('Micro Recall: {:.2f}'.format(recall_score(y_true, y_pred, average='micro')))
#     print('Micro F1-score: {:.2f}\n'.format(f1_score(y_true, y_pred, average='micro')))

#     print('Macro Precision: {:.2f}'.format(precision_score(y_true, y_pred, average='macro')))
#     print('Macro Recall: {:.2f}'.format(recall_score(y_true, y_pred, average='macro')))
#     print('Macro F1-score: {:.2f}\n'.format(f1_score(y_true, y_pred, average='macro')))

#     print('Weighted Precision: {:.2f}'.format(precision_score(y_true, y_pred, average='weighted')))
#     print('Weighted Recall: {:.2f}'.format(recall_score(y_true, y_pred, average='weighted')))
#     print('Weighted F1-score: {:.2f}'.format(f1_score(y_true, y_pred, average='weighted')))

    print('\nClassification Report\n')
    print(classification_report(y_true, y_pred, target_names=['Hub', 'Energy Management','Health Monitor','Camera','Appliances','Non IoT']))

Load dataset

In [5]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk("./data/tokenized_dataset_category")

In [12]:
datasets = tokenized_datasets.train_test_split(test_size=0.05,stratify_by_column="label",seed=42 )

Loading cached split indices for dataset at data/tokenized_dataset_category/cache-3770abc46fab0fd4.arrow and data/tokenized_dataset_category/cache-c63dfbd9e22fe980.arrow


In [13]:
d_train = datasets['test'].train_test_split(test_size=0.2,stratify_by_column='label',seed=42)

In [14]:
d_test = d_train['test'].train_test_split(test_size=0.5,stratify_by_column='label',seed=42)

In [15]:
from datasets.dataset_dict import DatasetDict
dd = DatasetDict({"train":d_train['train'],"test":d_test['train'],"validation":d_test['test']})

In [16]:
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 116124
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 14516
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 14516
    })
})

Load pretrained models

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./models", max_len=576)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file ./models/config.json
Model config RobertaConfig {
  "_name_or_path": "./models",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

Didn't find file ./models/tokenizer.json. We won't load it.
Didn't find file ./models/added_tokens.json. We won't load it.
Didn't find file ./models/special_tokens_

### model1-Test Trainer 

In [21]:
from transformers import AutoModelForSequenceClassification

model1 = AutoModelForSequenceClassification.from_pretrained("test_trainer/checkpoint-21780",num_labels=6)

loading configuration file test_trainer/checkpoint-21780/config.json
Model config RobertaConfig {
  "_name_or_path": "test_trainer/checkpoint-21780",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 578,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "

In [24]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model1.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(578, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [35]:
from transformers import TextClassificationPipeline

pipeline1 = TextClassificationPipeline(model=model1,tokenizer=tokenizer, task='iot-classification',device=0)

In [37]:
pipeline1(tokenized_datasets['text'][5000])

[{'label': 'LABEL_3', 'score': 0.5416531562805176}]

In [38]:
tokenized_datasets['label'][5000]

3

In [39]:
from transformers import TrainingArguments, Trainer

training_args1 = TrainingArguments(
    output_dir="test_trainer/checkpoint-21780",
    learning_rate=1e-5,
    evaluation_strategy="epoch",    
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_strategy="epoch",
    gradient_accumulation_steps=8,
    seed=42

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [40]:
trainer1 = Trainer(
    model=model1,
    args=training_args1,
#     train_dataset=dd['train'],
#     eval_dataset=dd['validation']
#     compute_metrics=compute_metrics,
)

In [41]:
pred1 = trainer1.predict(dd['test'])

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 14516
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [43]:
compute_metrics(pred1)

{'accuracy': 0.8146183521631304}

### model2-Cat Trainer 1

In [44]:
from transformers import AutoModelForSequenceClassification

model2 = AutoModelForSequenceClassification.from_pretrained("cat_trainer_1/checkpoint-816",num_labels=6)

loading configuration file cat_trainer_1/checkpoint-816/config.json
Model config RobertaConfig {
  "_name_or_path": "cat_trainer_1/checkpoint-816",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 578,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "fl

In [46]:
from transformers import TrainingArguments, Trainer

training_args2 = TrainingArguments(
    output_dir="cat_trainer_1/checkpoint-816",
    learning_rate=1e-5,
    evaluation_strategy="epoch",    
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_strategy="epoch",
    gradient_accumulation_steps=8,
    seed=42

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
trainer2 = Trainer(
    model=model2,
    args=training_args2,
#     train_dataset=dd['train'],
#     eval_dataset=dd['validation']
#     compute_metrics=compute_metrics,
)

In [49]:
pred2 = trainer2.predict(dd['test'])

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 14516
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [50]:
compute_metrics(pred2)

{'accuracy': 0.8224717553044916}

### model3-Cat Trainer 2

In [51]:
from transformers import AutoModelForSequenceClassification

model3 = AutoModelForSequenceClassification.from_pretrained("cat_trainer_2/checkpoint-5440",num_labels=6)

loading configuration file cat_trainer_2/checkpoint-5440/config.json
Model config RobertaConfig {
  "_name_or_path": "cat_trainer_2/checkpoint-5440",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 578,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "

In [52]:
from transformers import TrainingArguments, Trainer

training_args3 = TrainingArguments(
    output_dir="cat_trainer_2/checkpoint-5440",
    learning_rate=1e-5,
    evaluation_strategy="epoch",    
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_strategy="epoch",
    gradient_accumulation_steps=8,
    seed=42

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [53]:
trainer3 = Trainer(
    model=model3,
    args=training_args3,
#     train_dataset=dd['train'],
#     eval_dataset=dd['validation']
#     compute_metrics=compute_metrics,
)

In [54]:
pred3 = trainer3.predict(dd['test'])

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 14516
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [55]:
compute_metrics(pred3)

{'accuracy': 0.8227473133094516}

### model4-Cat Trainer 3

In [56]:
from transformers import AutoModelForSequenceClassification

model4 = AutoModelForSequenceClassification.from_pretrained("cat_trainer_3/checkpoint-3171",num_labels=6)

loading configuration file cat_trainer_3/checkpoint-3171/config.json
Model config RobertaConfig {
  "_name_or_path": "cat_trainer_3/checkpoint-3171",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 578,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "

In [57]:
from transformers import TrainingArguments, Trainer

training_args4 = TrainingArguments(
    output_dir="cat_trainer_3/checkpoint-3171",
    learning_rate=1e-5,
    evaluation_strategy="epoch",    
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_strategy="epoch",
    gradient_accumulation_steps=8,
    seed=42

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [58]:
trainer4 = Trainer(
    model=model4,
    args=training_args4,
#     train_dataset=dd['train'],
#     eval_dataset=dd['validation']
#     compute_metrics=compute_metrics,
)

In [59]:
pred4 = trainer4.predict(dd['test'])

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 14516
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [60]:
compute_metrics(pred4)

{'accuracy': 0.8140672361532103}

Result

In [61]:
print(compute_metrics(pred1))
print(compute_metrics(pred2))
print(compute_metrics(pred3))
print(compute_metrics(pred4))


{'accuracy': 0.8146183521631304}
{'accuracy': 0.8224717553044916}
{'accuracy': 0.8227473133094516}
{'accuracy': 0.8140672361532103}
